In [3]:
#SortUnifk Sorter 
#Mit Debugging-Struktur 

import math
import random 
import numpy as np
%run SortDet.ipynb

#Hilfsfunktion: Berechnet Arraykosten 
def compute_cost(r):
    """Berechnet cost′(r) = Summe der |r[i+1] - r[i]| für i = 0 bis n-2"""
    cost = 0.0
    for i in range(len(r) - 1):
        if r[i] is not None and r[i+1] is not None:
            cost += abs(r[i+1] - r[i])
    return cost

#Rekursionsanker. Sorter ruft in den Buckets und im Backyard SortDet auf.
class SortUnif1:
    def __init__(self, A, start, end, alpha, beta, label="SortUnif1"):
        self.A = A
        self.start = start
        self.end = end
        self.n = end - start
        self.alpha = alpha
        self.beta = beta
        self.label = label

        self.M = max(1, int(self.n ** alpha))   # Anzahl Buckets
        self.B = int(self.n ** beta)            # Größe Backyard
        self.N = self.n - self.B                # Vorderer Arrayteil (für Buckets)
        self.C = max(1, self.N // self.M)       # Kapazität pro Bucket

        # Erzeugt gleichmäßige Bucketstruktur (Kapazität +/- 1) im vorderen Arrayteil N
        base_c = self.N // self.M
        remainder = self.N % self.M
        s = self.start
        self.bucket_ranges = []
        for i in range(self.M):
            c = base_c + (1 if i < remainder else 0)
            e = s + c
            self.bucket_ranges.append((s, e))
            s = e

        # Jedes Bucket bekommt eigenen SortDet-Sortierer 
        self.bucket_sorters = [
            SortDet(A, s, e, label=f"{self.label}.b{i}")
            for i, (s, e) in enumerate(self.bucket_ranges)
        ]

        self.bucket_fill = [0] * self.M
        self.bucket_capacities = [e - s for (s, e) in self.bucket_ranges]
        self.interval_len = 1 / self.M

        # Aufbau Backyard
        self.backyard_start = self.start + self.N
        self.backyard_end = self.end
        self.backyard_sorter = SortDet(
            A, self.backyard_start, self.backyard_end,
            label=f"{self.label}.backyard"
        )
        
    def insert_with_original(self, x_prime, original_x):
        x_prime = max(x_prime, 1e-8)
        h = min(math.ceil(x_prime * self.M), self.M)
        bucket_id = h - 1

        if self.bucket_fill[bucket_id] < self.bucket_capacities[bucket_id]:
            success = self.bucket_sorters[bucket_id].insert(original_x)
            if success:
                self.bucket_fill[bucket_id] += 1
                return True

        return self.backyard_sorter.insert(original_x)

    def insert(self, x):
        return self.insert_with_original(x, x)

#Für Debugging, sonst ignorieren.   
class SortUnif1Wrapper:
    def __init__(self, n, alpha, beta):
        self.n = n
        self.A = [None] * n
        self.alpha = alpha
        self.beta = beta 
        self.sorter = SortUnif1(self.A, 0, n, alpha, beta)

    def insert(self, x):
        return self.sorter.insert(x)

    def get_state(self):
        return self.A

#Eigentlicher rekursiver Sortierer nach Abrahamsen et al. 
#Bricht allerdings bei Fail(Emergency) nicht ab, sondern fügt übrig gebliebene Elemente in freie Arrayzellen.     
class SortUnifk:
    def __init__(self, A, start, end, alpha, beta, k, label="root", enable_emergency_at_root=False, is_root=True):
        self.A = A
        self.start = start
        self.end = end
        self.n = end - start
        self.alpha = alpha
        self.beta = beta
        self.k = k
        self.label = label 

        self.is_root = is_root
        self.enable_emergency_at_root = enable_emergency_at_root and is_root #Emergency Fill 
        self.emergency_active = False #Dann wie im Paper von Abrahamsen et al.
        self._em_range_idx = 0
        self._em_pos_in_range = None

        # Anzahl Buckets
        self.M = max(1, int(self.n ** alpha))  
        # Backyard-Größe
        self.B = int(self.n ** beta)  
        # Vorderer Arrayteil (für Buckets)
        self.N = self.n - self.B   
        # Bucket Kapazität
        self.C = max(1, self.N // self.M)  
        # Länge Werteintervall 
        self.interval_len = 1 / self.M

        #Gleichmäßiges Aufspannen von Buckets (+/- 1 Kapazität)
        base_c = self.N // self.M
        remainder = self.N % self.M
        s = start
        self.bucket_ranges = []
        for i in range(self.M):
            c = base_c + (1 if i < remainder else 0)
            e = s + c
            self.bucket_ranges.append((s, e))
            s = e

        #Baue pro Bucket den zuständigen (Unter-)Sortierer SortUnif_k-1 usw. bis SortUnif_1
        self.bucket_sorters = []
        for i, (s, e) in enumerate(self.bucket_ranges):
            sublabel = f"{label}.b{i}"
            if k == 1:
                #SU1 ohne Emergency
                self.bucket_sorters.append(SortUnif1(A, s, e, alpha, beta, label=sublabel))
            else:
                #SU1 mit Emergency
                self.bucket_sorters.append(SortUnifk(A, s, e, alpha, beta, k - 1, label=sublabel, 
                                                     enable_emergency_at_root=False, is_root=False))

        self.bucket_fill = [0] * self.M
        self.bucket_capacities = [end - start for (start, end) in self.bucket_ranges]

        #Backyard Aufbau
        self.backyard_start = self.N + start
        self.backyard_end = end
        self.backyard_sorter = SortDet(A, self.backyard_start, self.backyard_end, label=f"backyard@{self.backyard_start}:{self.backyard_end}")


        #Emergency-Cursor für Emergency Fill vorbereiten (nur für root)
        if self.bucket_ranges:
            self._em_pos_in_range = self.bucket_ranges[0][0]
        else:
            self._em_pos_in_range = self.start

    # Emergency Fall: Findet erste freie Zelle im vorderern Array-Teil
    def _emergency_find_slot(self):
        while self._em_range_idx < len(self.bucket_ranges):
            s, e = self.bucket_ranges[self._em_range_idx]
            pos = max(self._em_pos_in_range, s)
            while pos < e and self.A[pos] is not None:
                pos += 1
            if pos < e:
                self._em_pos_in_range = pos + 1
                return pos
            self._em_range_idx += 1
            if self._em_range_idx < len(self.bucket_ranges):
                self._em_pos_in_range = self.bucket_ranges[self._em_range_idx][0]
        return None
        

    def _emergency_insert(self, original_x):
        slot = self._emergency_find_slot()
        if slot is None:
            return False
        self.A[slot] = original_x
        #Aktualisiert den Füllstand des Buckets, wichtig für Debugging
        for b_id, (s, e) in enumerate(self.bucket_ranges):
            if s <= slot < e:
                if self.bucket_fill[b_id] < self.bucket_capacities[b_id]:
                    self.bucket_fill[b_id] += 1
                break
        return True

    #Mapping/Hashing 
    def get_bucket_id(self, x):
        h = min(math.ceil(x * self.M), self.M)
        return h - 1

    #Übergibt x als original_x in die Rekursion
    def insert(self, x):
        return self.insert_with_original(x, x)

    #Einfügelogik
    #x_prime  = "lokale" Koordinate in [0,1] für das aktuelle Rekursionslevel (für Bucket-Wahl in Rekursion)
    #original_x = ursprünglicher Wert, wird unverändert bis zu untersten Ebene weitergergegeben
    def insert_with_original(self, x_prime, original_x):

        # Falls Phase 3: Emergency einfügen 
        if self.enable_emergency_at_root and self.emergency_active:
            return self._emergency_insert(original_x)
        
        # Phase 1: regulär im Ziel-Bucket (rekursiv) einfügen
        x_prime = max(x_prime, 1e-8) #kleine Untergrenze für Randfälle
        h = min(math.ceil(x_prime * self.M), self.M)
        bucket_id = h - 1
        x_prime_next = x_prime * self.M - h + 1 #Re-Skalierung für nächstes Rekursionslevel

        if self.bucket_fill[bucket_id] < self.bucket_capacities[bucket_id]:
            sorter = self.bucket_sorters[bucket_id]
            success = sorter.insert_with_original(x_prime_next, original_x)

            if success:
                self.bucket_fill[bucket_id] += 1
                return True

        # Phase 2: Top-Level-Backyard als Überlaufpuffer
        if self.backyard_sorter.insert(original_x):
            #print(f"[k={self.k}]  → x={original_x:.4f} geht in den Backyard")
            return True
        
        #Backyard voll → Phase 3 aktivieren (nur root)
        if self.enable_emergency_at_root:
            self.emergency_active = True
            return self._emergency_insert(original_x)
        return False

#Wrapper für Debugging 
class SortUnifkWrapper:
    def __init__(self, n, alpha, beta, k, enable_emergency=False):
        self.n = n
        self.A = [None] * n
        self.alpha = alpha
        self.beta = beta
        self.k = k
        self.sorter = SortUnifk(self.A, 0, n, alpha, beta, k, 
                               enable_emergency_at_root=enable_emergency,
                                is_root=True)

    def insert(self, x):
        return self.sorter.insert(x)

    def get_state(self):
        return self.A

#Generiert unterschiedlcihe Eingabeverteilungen 
def generate_data(n, distribution="uniform"):
    if distribution == "uniform":
        return [random.uniform(1e-8, 1) for _ in range(n)]
    elif distribution == "triangular":
        return [random.triangular(0, 1, 0.5) for _ in range(n)]
    elif distribution == "beta":
        return [np.random.beta(a=2, b=5) for _ in range(n)]
    elif distribution == "normal":
        return [min(max(random.gauss(0.5, 0.15), 0), 1) for _ in range(n)]
    else:
        raise ValueError(f"unbekannte distribution: {distribution}")

# Führt SortUnifk auf Daten aus
def run_sortunifk_on_data(data, alpha=1/3, beta=2/3, k=3, enable_emergency=True):
    n = len(data)
    W = SortUnifkWrapper(n, alpha, beta, k, enable_emergency=enable_emergency)
    ok_all = True
    for x in data:
        if not W.insert(x):
            ok_all = False
            break
    A = W.get_state()
    filled = sum(1 for v in A if v is not None)
    res = {
        "ok_all": ok_all,
        "filled": filled,
        "emergency_active": getattr(W.sorter, "emergency_active", False),
        "cost": compute_cost(A),
        "A": A
    }
    return res